Guidance on using this dataset: https://www.kaggle.com/code/acelevin/identifying-playing-cards

Download dataset from: https://www.kaggle.com/datasets/gunhcolab/object-detection-dataset-standard-52card-deck/data

In [4]:
import torch
import torchvision
from PIL import Image
from torch.utils.data import Dataset
import os
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [27]:
#hyper parameters:
RANDOM_SEED = 123
BATCH_SIZE = 32
NUM_EPOCHS = 256

In [24]:
# Load in data
train_labels = pd.read_csv("data/train_labels.csv")
test_labels = pd.read_csv("data/test_labels.csv")

# change class labels from str to int
name_to_int_dict = {'ace of spades': 0, 'two of spades':1, 'three of spades':2, 'four of spades':3, 
                    'five of spades':4, 'six of spades':5, 'seven of spades':6, 'eight of spades':7, 
                    'nine of spades':8, 'ten of spades':9, 'jack of spades':10, 'queen of spades':11, 
                    'king of spades':12, 'ace of hearts': 13, 'two of hearts': 14, 'three of hearts': 15, 
                    'four of hearts': 16, 'five of hearts': 17, 'six of hearts': 18, 'seven of hearts':19, 
                    'eight of hearts':20, 'nine of hearts':21, 'ten of hearts':22, 'jack of hearts': 23, 
                    'queen of hearts': 24, 'king of hearts':25, 'ace of clubs': 26, 'two of clubs': 27, 
                    'three of clubs': 28, 'four of clubs': 29, 'five of clubs': 30, 'six of clubs': 31,
                    'seven of clubs':32, 'eight of clubs':33, 'nine of clubs':34, 'ten of clubs':35, 
                    'jack of clubs': 36, 'queen of clubs': 37, 'king of clubs':38, 'ace of diamonds': 39, 
                    'two of diamonds': 40, 'three of diamonds': 41, 'four of diamonds': 42, 'five of diamonds': 43, 
                    'six of diamonds': 44,'seven of diamonds':45, 'eight of diamonds':46, 'nine of diamonds':47, 
                    'ten of diamonds':48, 'jack of diamonds': 49, 'queen of diamonds': 50,'king of diamonds':51,
                    
                    # these are to account for some typos in the dataset
                    'three of dimaonds': 41, 'four of dimaonds': 42, 'five of dimaonds': 43, 'six of dimaonds': 44, 
                    'eigth of clubs':33, 'seven of seven':32 
                    }

test_labels['class_number'] = test_labels['class'].map(name_to_int_dict)
train_labels['class_number'] = train_labels['class'].map(name_to_int_dict)

# These should print empty to show that all labels are accounted for
print(test_labels.loc[test_labels['class_number'].isnull()])
print(train_labels.loc[train_labels['class_number'].isnull()])

Empty DataFrame
Columns: [filename, width, height, class, xmin, ymin, xmax, ymax, class_number]
Index: []
Empty DataFrame
Columns: [filename, width, height, class, xmin, ymin, xmax, ymax, class_number]
Index: []
